# CLIP order images - Notebook

We order Macron caricatures using boolean CLIP filters.  

## CLIP propositions

This notebook applies boolean filters depending on CLIP VitL-14.  
Each image is analyzed and two propositions are submitted. The winning proposition is returned (or a boolean if the first proposition is True).  

Eg.  We submit a cat image, and two propositions: "image of a cat", "image of a dog".  
CLIP will return "image of a cat", or True (as the first proposition is True).  

## Macron caricatures filtering

We submit a list of propositions and retain all images that check all filters (all boolean filter results equal to True).  
In the end, we go from 300 images to a short-list of 10 images, with a noticeable resemblance to Macron while beeing caricatures.  

# Import and install libs

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.auto import tqdm

tqdm.pandas()

## Import default CLIP test snippet

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

# Make functions to perform CLIP on image

In [ ]:
def labels_and_probs_to_answer(label_list, probs):
    
    assert len(label_list) == 2, 'label_list must be of len 2'
        
    some_floats = probs.detach().numpy()
    np_array = (np.rint(some_floats)).astype(int)
    prob_list = np_array.tolist()[0]
    
    return label_list[not prob_list[0]]

In [ ]:
# returns winning proposition

def from_filename_to_answer(filename, label_list):
    with Image.open(filename) as im:
        im = im.convert("RGB")
        # display(im)
        inputs = processor(text=label_list, images=im, return_tensors="pt", padding=True)

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
    
    return labels_and_probs_to_answer(label_list, probs)

In [ ]:
# returns True if first proposition has won

def from_filename_to_bool(filename, label_list):
    with Image.open(filename) as im:
        im = im.convert("RGB")
        # display(im)
        inputs = processor(text=label_list, images=im, return_tensors="pt", padding=True)

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
    
    return labels_and_probs_to_answer(label_list, probs) == label_list[0]

## Test functions

In [ ]:
# output should be "picture of emmanuel macron" and "True"
# no assertions to prevent from blocking future runs

print(from_filename_to_answer('image_to_image/macron/macron.png', ['picture of emmanuel macron', 'picture of a man']))
print(from_filename_to_bool('image_to_image/macron/macron.png', ['picture of emmanuel macron', 'picture of a man']))

# RUN

## Load filename list and convert to dataframe

In [ ]:
input_path = "image_to_image/macron_db"
filename_list = sorted(glob(input_path+"/*/*", recursive = True))
# filename_list

In [ ]:
macron_db_df = pd.DataFrame(filename_list)
macron_db_df.columns = ['filename']
# macron_db_df

## Apply CLIP filters

In [ ]:
# obsolete use of the winning proposition

# macron_db_df['caricature'] = macron_db_df.progress_apply(lambda row: from_filename_to_answer(row['filename'], label_list), axis=1)

In [ ]:
# current use of the boolean winning proposition
label_list = ["looks like a caricature", "looks like a random drawing"]
macron_db_df['is_caricature'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

In [ ]:
# perform CLIP filtering on other questions

# DEACTIVATED FILTER: too much randomness
# label_list = ["this picture is a meme", "this picture is not a meme"]
# macron_db_df['is_meme'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["a black and white image", "a color photograph"]
macron_db_df['is_black_and_white_image'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["a drawing of a portrait", "a picture of a portrait"]
macron_db_df['is_drawing'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["a meme about emmanuel macron", "a meme about you dont say"]
macron_db_df['is_macron_meme'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["a man with two eyes", "a man missing an eye"]
macron_db_df['has_two_eyes'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["a beautiful portrait", "an ugly portrait"]
macron_db_df['is_beautiful_portrait'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

label_list = ["looks like a meme", "looks like a random drawing"]
macron_db_df['looks_like_meme'] = macron_db_df.progress_apply(lambda row: from_filename_to_bool(row['filename'], label_list), axis=1)

In [ ]:
macron_db_df

In [ ]:
# save results to csv

macron_db_df.to_csv("macron_db_df.csv", sep=',')

## Filter on CLIP filter results

In [ ]:
# filter results on CLIP filter output

macron_db_df_bestof = macron_db_df[macron_db_df.is_caricature]
# macron_db_df_bestof = macron_db_df_bestof[macron_db_df.is_meme]
macron_db_df_bestof = macron_db_df_bestof[macron_db_df.is_black_and_white_image]
macron_db_df_bestof = macron_db_df_bestof[macron_db_df.is_drawing]
macron_db_df_bestof = macron_db_df_bestof[macron_db_df.is_macron_meme]

macron_db_df_bestof = macron_db_df_bestof[macron_db_df.has_two_eyes]
macron_db_df_bestof = macron_db_df_bestof[macron_db_df.is_beautiful_portrait]
macron_db_df_bestof = macron_db_df_bestof[macron_db_df.looks_like_meme]

macron_db_df_bestof

In [ ]:
# save to csv
macron_db_df_bestof.to_csv("macron_db_df_bestof.csv", sep=',')

# resume from csv
macron_db_df_bestof = pd.read_csv("macron_db_df_bestof.csv", sep=',', index_col=0)
macron_db_df_bestof

## Move best pictures to folder and display images here

In [ ]:
# move files into bestof folder

for filename in macron_db_df_bestof['filename']:
    !cp $filename bestof/

In [ ]:
# display bestof files

for filename in macron_db_df_bestof['filename']:
     print(filename)
     with Image.open(filename) as im:
        im = im.convert("RGB")
        display(im)